<h1>Clustering Toronto Neighborhood</h1>

<h1>Queston 1</h1>

import libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('geopy imported.')



In [ ]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
print('folium imported.')

import folium # map rendering library

In [ ]:
!conda install -c conda-forge BeautifulSoup4 --yes
print('BeautifulSoup4 imported.')

from bs4 import BeautifulSoup

print('All Libraries imported.')

Get Data from wikipedia

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

rows = soup.find('table', class_='wikitable sortable').find_all('tr')

PostalCode = []
Borough = []
Neighborhood = []

for row in rows:
    line = row.find_all('td')
    #print(cell)
    if len(line) > 0:
        PostalCode.append(line[0].text)
        Borough.append(line[1].text)
        Neighborhood.append(line[2].text.rstrip('\n'))

wikidataf = pd.DataFrame({"Postal Code": PostalCode,
                        "Borough": Borough,
                        "Neighborhood": Neighborhood})

wikidataf.head()

Data Clean Up

In [ ]:
wikidataf2 = wikidataf[wikidataf.Borough != "Not assigned"]
wikidataf3 = wikidataf2.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
for a, b in wikidataf3.iterrows():
    if b["Neighborhood"] == "Not assigned":
        b["Neighborhood"] = b["Borough"]
        
wikidataf3.head(10)
wikidataf3.shape

<h2>Question 2</h2>


Loading CSV

In [ ]:
geod = pd.read_csv("https://cocl.us/Geospatial_data")

wikidataf4 = wikidataf3.merge(geod, on="Postal Code", how="left")
wikidataf4.head()

<h2>Question 3</h2>

Get GEO Coordinates

In [ ]:

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Plot Map

In [ ]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(wikidataf4['Latitude'], wikidataf4['Longitude'], wikidataf4['Borough'], wikidataf4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
Filter by boroughs that contain the word Toronto

In [ ]:

toronto_data = wikidataf4[wikidataf4['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

Creating Map of Boroughs that contain Toronto word

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

Define Foursquare Credentials

In [ ]:

CLIENT_ID = 'VVHKZVYCQSV3QIGQ2UB2ELR4RAC2YBU1FXHBTSNR5UVGOKYW' # your Foursquare ID
CLIENT_SECRET = 'EYWG5ZQT2KPEV2REO3HFJDP5NISVVV2E2N13Z51Z02JOA5I3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

get the top 100 venues that are within a radius of 500 meters

In [ ]:
:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
venues_list=[]

# create URL
for pos, lat, lng, borough, neighborhood in zip(toronto_data['Postal Code'], toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for result in results:
        venues_list.append((
                pos,
                borough,
                neighborhood,
                lat, 
                lng, 
                result['venue']['name'],
                result['venue']['location']['lat'], 
                result['venue']['location']['lng'],  
                result['venue']['categories'][0]['name']))

venuesdf = pd.DataFrame(venues_list)
venuesdf.columns = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLat', 'VenueLong', 'VenueCat']
venuesdf.head()

check the size of the resulting dataframe

In [ ]:
print (venuesdf.shape)

In [ ]:

venuesdf.groupby('Borough').count()

print('There are {} uniques categories.'.format(len(venuesdf['VenueCat'].unique())))


group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venuesdf['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

write a function to sort the venues

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Cluster Neighborhoods into 4 clusters

In [ ]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Cluster Visualization and Examination

In [ ]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]